# Test

In [1]:
import molsysmt as msm

In [2]:
import openmm as mm
from openmm import app
from openmm import unit

In [3]:
molsys = msm.convert('4POC')

In [4]:
molsys = msm.extract(molsys, selection='molecule_type=="protein"')

In [5]:
molsys = msm.build.add_missing_hydrogens(molsys)

In [6]:
molsys = msm.build.solvate(molsys)

In [7]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [8]:
topology = msm.convert(molsys, 'openmm.Topology')

In [9]:
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)

In [10]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

In [11]:
platform = mm.Platform.getPlatformByName("CPU")

In [12]:
simulation = app.Simulation(topology, system, integrator, platform)

In [13]:
coordinates = msm.get(molsys, coordinates=True)
positions = msm.pyunitwizard.convert(coordinates[0], to_form='openmm.unit')

In [14]:
simulation.context.setPositions(positions)
simulation.minimizeEnergy()

In [15]:
from molsysmt.thirds.openmm.reporters import MolSysMTTrajectoryDictReporter

In [16]:
reporter = MolSysMTTrajectoryDictReporter(10)

In [17]:
simulation.reporters.append(reporter)

In [18]:
simulation.step(100)

In [19]:
traj = reporter.finalize()

In [20]:
traj

{'time': Quantity(value=array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ]), unit=picosecond),
 'coordinates': Quantity(value=array([[[-0.45974578, -1.7492454 , -0.41946606],
         [-0.54349788, -1.69396129, -0.40805317],
         [-0.44343834, -1.80360854, -0.33592137],
         ...,
         [ 5.71245086, -0.83405943,  2.776235  ],
         [-1.92531895, -2.0580962 ,  3.29697089],
         [ 5.83888242, -1.61575717,  5.65570512]],
 
        [[ 2.38045406,  6.28400774, -0.41962602],
         [ 2.29794749,  6.34079759, -0.40664218],
         [ 2.39939293,  6.22837663, -0.33748285],
         ...,
         [ 0.03252015,  7.19882329,  2.77531392],
         [ 0.9143502 ,  5.97455063,  3.29787384],
         [ 0.15883804,  6.41684034,  5.65552087]],
 
        [[ 2.38009286,  6.28396525, -0.41958003],
         [ 2.29745588,  6.34047984, -0.40622964],
         [ 2.39935355,  6.22379059, -0.34078257],
         ...,
         [ 0.0310489 ,  7.19953919,  2.77491244],
     

In [25]:
simulation.integrator.getStepSize()

Quantity(value=0.002, unit=picosecond)

In [27]:
int(0.6)

0